# Flujo Máximo: Ford Fulkerson

Para probar que la implementación de nuestro paquete sea correcta, usaremos datos de este [repositorio](https://github.com/SumitPadhiyar/parallel_ford_fulkerson_gpu/tree/master/dataset), donde hacen una implementación del algoritmo de Ford Fulkerson usando gpu's. En un inicio se intento usar los datos de la [página](http://elib.zib.de/pub/mp-testdata/) propuesta por el profesor pero notamos que al declarar los nodos y los arcos teníamos ciertos errores, por lo que preferimos buscar nuevos datos.

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import ffmaxflow as ff
import time

In [6]:
!pip install ../ffmaxc/

Processing /home/diramtz/Documents/Optimizacion/ProyectoFinal_MaxFlow/ffmaxc
  Created wheel for UNKNOWN: filename=UNKNOWN-0.0.0-cp38-cp38-linux_x86_64.whl size=250171 sha256=41d49be01f87a5b3eb896360117c1b6fdecefdd07686faba7fbec544f319c503
  Stored in directory: /tmp/pip-ephem-wheel-cache-vgapb55w/wheels/64/ec/00/07fbc8f81d14bd6e084a678ffbbd60843bf4aaa867d0489e69
Successfully built UNKNOWN
  Attempting uninstall: UNKNOWN
    Found existing installation: UNKNOWN 0.0.0
    Uninstalling UNKNOWN-0.0.0:
      Successfully uninstalled UNKNOWN-0.0.0


## Leer los datos

In [22]:
#with open("../data/elist1440.txt") as myfile:
with open("../data/elist960d.txt") as myfile:
    meta = [next(myfile)[:-1] for x in range(4)]

In [23]:
print("Número de nodos: {}". format(int(meta[0])))
print("Número de arcos: {}". format(int(meta[1])))
print("Nodo fuente: {}". format(int(meta[2])))
print("Nodo destino: {}". format(int(meta[3])))

Número de nodos: 960
Número de arcos: 9488
Nodo fuente: 1
Nodo destino: 960


In [24]:
#df = pd.read_csv("../data/elist1440.txt",skiprows = 4,  sep = "\s+")
df = pd.read_csv("../data/elist960d.txt",skiprows = 4,  sep = "\s+")

In [25]:
data = df.values.astype(int)

In [26]:
data.shape

(9487, 3)

In [27]:
data

array([[  1,  65, 800],
       [  1, 273,   0],
       [  1, 305,   2],
       ...,
       [960, 624,   5],
       [960, 896, 800],
       [960, 944, 800]])

## Paquete original

In [54]:
red = ff.create_flow_network()

In [55]:
for row in range(data.shape[0]):
    
    if data[row,0] == int(meta[2]):
        red.create_vertex('{}'.format(data[row,0]), True, False)
    
    elif data[row,1] == int(meta[3]):
        red.create_vertex('{}'.format(data[row,1]), False, True)
    
    else:
        red.create_vertex('{}'.format(data[row,0]), False, False)
        red.create_vertex('{}'.format(data[row,1]), False, False)
        
    red.create_edge('{}'.format(data[row,0]), '{}'.format(data[row,1]), data[row,2])

Nodo destino ya ha sido agregado. 
 El cálculo del flujo máximo continúa con el primer valor asignado al nodo destino.
Nodo destino ya ha sido agregado. 
 El cálculo del flujo máximo continúa con el primer valor asignado al nodo destino.
Nodo destino ya ha sido agregado. 
 El cálculo del flujo máximo continúa con el primer valor asignado al nodo destino.
Nodo destino ya ha sido agregado. 
 El cálculo del flujo máximo continúa con el primer valor asignado al nodo destino.
Nodo destino ya ha sido agregado. 
 El cálculo del flujo máximo continúa con el primer valor asignado al nodo destino.
Nodo destino ya ha sido agregado. 
 El cálculo del flujo máximo continúa con el primer valor asignado al nodo destino.


In [56]:
red.MaxFlow()

0

## Con paquetería networkx

In [31]:
G = nx.DiGraph()

In [32]:
for row in range(data.shape[0]):
    G.add_edge('{}'.format(data[row,0]), '{}'.format(data[row,1]), capacity = data[row,2])

In [34]:
flow_value, flow_dict = nx.maximum_flow(G, '{}'.format(int(meta[2])), '{}'.format(int(meta[3])))

flow_value

## Con paquete reimplementado

In [18]:
import ffmaxc as ffc

In [19]:
red = ffc.create_flow_network()

In [20]:
for row in range(data.shape[0]):
    
    if data[row,0] == int(meta[2]):
        red.create_vertex('{}'.format(data[row,0]), True, False)
    
    elif data[row,1] == int(meta[3]):
        red.create_vertex('{}'.format(data[row,1]), False, True)
    
    else:
        red.create_vertex('{}'.format(data[row,0]), False, False)
        red.create_vertex('{}'.format(data[row,1]), False, False)
        
    red.create_edge('{}'.format(data[row,0]), '{}'.format(data[row,1]), data[row,2])

Nodo destino ya ha sido agregado. 
 El cálculo del flujo máximo continúa con el primer valor asignado al nodo destino.


AttributeError: 'NoneType' object has no attribute 'name'

In [21]:
red.MaxFlow()

'La red no tiene nodo origen y/o destino '

In [ ]:
# no sé por qué no funciona arriba si el ejemplo de abajo sí sale :( 
# lo checo :(

In [22]:
start_time = time.time()

red = ffc.create_flow_network()

red.create_vertex('o', True, False) 
red.create_vertex('t', False, True) 
red.create_vertex('a', False, False)
red.create_vertex('b', False, False)
red.create_vertex('c', False, False)
red.create_vertex('d', False, False)
red.create_vertex('e', False, False)

red.create_edge('o', 'a', 5)
red.create_edge('o', 'b', 7)
red.create_edge('o', 'c', 4)

red.create_edge('a', 'b', 1)
red.create_edge('a', 'd', 3)

red.create_edge('b', 'c', 2)
red.create_edge('b', 'd', 4)
red.create_edge('b', 'e', 5)

red.create_edge('c', 'e', 4)

red.create_edge('d', 't', 9)

red.create_edge('e', 'd', 1)
red.create_edge('e', 't', 9)

red.MaxFlow()

end_time = time.time()

In [23]:
red.MaxFlow()

15.0